# Obtain and Append Data

This notebook contains code to obtain and append financial data to avoid going over API request limits.

In [1]:
import pandas as pd
from datetime import date
from datetime import timedelta
import plotly.graph_objects as go

import requests
import json
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as ticker

import numpy as np
from IPython.display import clear_output
import time
# Set plot space as inline for inline plots and qt for external plots
%matplotlib inline

In [2]:
import platform
print(platform.python_version())

3.8.3


In [3]:
# Obtain the API credentials from an external hidden file

from passwords import api_key

In [4]:
# A function to obtain price history

#Utilizing "UniBit API" by Vincent Jiang
#Site: https://unibit.ai/product

#Errors may occur if credits run out, add an error code check to this function

def retrieve_ticker_price_info(ticker, start_date, end_date = False):
    
    # Obtain today's date
    today = date.today()
    
    # A condition to set the end date to today if there is no end date given
    if not end_date:
        end_date = today
    
    
    url = 'https://api.unibit.ai/v2/stock/historical/?tickers={}&interval=1&startDate={}&endDate={}&selectedFields=all&dataType=json&accessKey={}'.format(ticker, start_date, end_date, api_key)
    
    response = requests.get(url)
    
    if response.status_code != 200:
        print("The response status code is: " + str(response.status_code))
        return None
    
    return response.json()
    

In [5]:
def retrieve_price_history_dataframe(ticker, ticker_price_history):
    
    data_frame = pd.DataFrame(ticker_price_history['result_data'][ticker])
    
    # Set the index to the 'date' column
    data_frame.set_index('date', inplace = True)
    
    data_frame.sort_index(inplace=True)
    
    return data_frame

In [8]:
today = date.today()
a_year_ago = str(date.today() - timedelta(days=1*365))

## Obtain Original Data from the API

In [9]:
# The S&P 500 company ticker is ^GSPC

# Obtain the price info
inx_info = retrieve_ticker_price_info('^GSPC', a_year_ago)

# Create and view the dataframe
inx_df = retrieve_price_history_dataframe('^GSPC', inx_info)
inx_df.head()

,volume,high,low,adj_close,close,open
date,,,,,,
2019-07-22,3003720000,2990.71,2976.65,2985.03,2985.03,2981.93
2019-07-23,3313660000,3005.90,2988.56,3005.47,3005.47,2994.74
2019-07-24,3428980000,3019.59,2996.82,3019.56,3019.56,2998.77
2019-07-25,3645270000,3016.31,2997.24,3003.67,3003.67,3016.26
2019-07-26,3257590000,3027.98,3012.59,3025.86,3025.86,3013.25


In [11]:
# Save the data as a csv
# Keep the index as it contains the dates
inx_df.to_csv('financial_data/GSPC.csv')

## Import a CSV of Time Series Data

This code assumes the data is in ascending order, if this is not the case then make the relevant adjustments. 

In [37]:
inx_df = pd.read_csv('financial_data/GSPC.csv', index_col = 'date')

# Convert index to date
# Comment this out if the index is already date type
inx_df.index = pd.to_datetime(inx_df.index)

inx_df.head()

,volume,high,low,adj_close,close,open
date,,,,,,
2019-07-22,3003720000,2990.71,2976.65,2985.03,2985.03,2981.93
2019-07-23,3313660000,3005.90,2988.56,3005.47,3005.47,2994.74
2019-07-24,3428980000,3019.59,2996.82,3019.56,3019.56,2998.77
2019-07-25,3645270000,3016.31,2997.24,3003.67,3003.67,3016.26
2019-07-26,3257590000,3027.98,3012.59,3025.86,3025.86,3013.25


In [33]:
# Obtain the most recent date
inx_df.index.max()

Timestamp('2020-06-30 00:00:00')

## Obtain Most Recent Data From The API

In [43]:
# Obtain the next day after the series ends, as a string
next_day_series_ends = str(inx_df.index.max().date() + timedelta(days=1))

In [ ]:
# The S&P 500 company ticker is ^GSPC

# Obtain the price info
inx_info = retrieve_ticker_price_info('^GSPC', a_year_ago)

# Create and view the dataframe
inx_df = retrieve_price_history_dataframe('^GSPC', inx_info)
inx_df.head()